In [1]:
import petl as etl
from config import RAW, FUNDED_ORGS_LIST, la_names

In [2]:
date_parser = etl.dateparser('%Y-%m-%d 00:00:00')

## Load data

### Arts Council England Investment Programme

In [3]:
ace_investment_programme = (
    etl.fromcsv(RAW / 'arts-council-investment-programme.csv')
    .rename({
            'Applicant Name': 'organisation',
            "Type of organisation\n(NPO/IPSO/Transfer)": 'Source'
            })
    .addfield('Period', '2023/26')
    .cut('organisation', 'Source', 'Local authority', 'Period')
)

### Arts Council England Project Grants

In [4]:
ace_project_grants = (
    etl.fromcsv(RAW / 'arts-council-project-grants.csv')
    .convert('Award date', lambda d: date_parser(d).year)
    .rename({
            'Recipient': 'organisation',
            'Award date': 'Period'
            })
    .addfield('Source', 'Project Grant')
    .cut('organisation', 'Source', 'Local authority', 'Period')
)

In [5]:
data = etl.cat(
    ace_investment_programme,
    ace_project_grants
).selectin(
    'Local authority', la_names
).selectnotin(
    'organisation', ['-']
).convert(
    'organisation', lambda x: x.strip()
).aggregate(
    ('organisation', 'Local authority', 'Source'),
    len,
    field="Number"
).sort(
    ['organisation', 'Local authority']
)

In [6]:
data.tocsv(FUNDED_ORGS_LIST)